In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "31"

In [2]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048,
    load_in_4bit = True,
    load_in_8bit = False,
    full_finetuning = False,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
Unsloth: Failed to patch Gemma3ForConditionalGeneration.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 04-23 19:04:46 [__init__.py:239] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.51.3. vLLM: 0.8.4.
   \\   /|    NVIDIA A16. Num GPUs = 17. Max memory: 14.542 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.6. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False,
    finetune_language_layers   = True,
    finetune_attention_modules = True,
    finetune_mlp_modules       = True,

    r = 8,
    lora_alpha = 8,
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


In [4]:
from datasets import load_dataset

ds = load_dataset("iamtarun/python_code_instructions_18k_alpaca")
dataset = ds["train"]

In [5]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [6]:
def formatting_prompts_func(row):
    
    # instruction = {
    #     "content": row["instruction"],
    #     "role": "user"
    # }
    
    # output = {
    #     "content": row["output"],
    #     "role": "assistant"
    # }

    # convos = [instruction, output]
    # texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]

    instruction = row["instruction"]
    output = row["output"]
    texts = f"<start_of_turn>user\n{instruction}<end_of_turn>\n<start_of_turn>model\n{output}<end_of_turn>\n"
    
    return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func)

Map:   0%|          | 0/18612 [00:00<?, ? examples/s]

In [7]:
print(dataset[0]["text"])

<start_of_turn>user
Create a function to calculate the sum of a sequence of integers.<end_of_turn>
<start_of_turn>model
# Python code
def sum_sequence(sequence):
  sum = 0
  for num in sequence:
    sum += num
  return sum<end_of_turn>



In [8]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 10,
        # max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
        dataset_num_proc=2,
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/18612 [00:00<?, ? examples/s]

In [9]:
from unsloth.chat_templates import train_on_responses_only

trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=64):   0%|          | 0/18612 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()